In [2]:
# Get the data for expeirment
import sklearn
import pandas as pd 
import numpy as np

from sklearn import tree
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.metrics import explained_variance_score, mean_squared_error
from scipy.spatial.distance import cdist
from numpy.linalg import pinv
from math import sqrt

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import geopy.distance as distance
from sklearn.preprocessing import MinMaxScaler

import Data_utils
import matplotlib.ticker as ticker
from functools import reduce
from collections import Counter

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.inspection import PartialDependenceDisplay
from scikeras.wrappers import KerasRegressor
from sklearn.metrics import r2_score

import plotly.express as px
import TsModel
from sklearn.utils import validation
from windrose import WindroseAxes

In [3]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [22]:
data = pd.read_csv(f'Data\processed_ab_no_precp_test.txt', encoding = "ISO-8859-1")

C:\Users\PPL\AppData\Local\Temp\ipykernel_4172\3793995334.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(f'Data\processed_ab_no_precp_test.txt', encoding = "ISO-8859-1")


In [23]:
data['date'] = pd.to_datetime(data['date'], errors='coerce')

In [24]:
# Initialize one-hot encoding
data['spring'] = 0
data['summber'] = 0
data['fall'] = 0
data['winter'] = 0

In [25]:
# Use meterologial season start and end dates
seasons = {
    'spring': [3,4,5],
    'summer': [6,7,8],
    'fall': [9,10,11],
    'winter': [12,1,2],
}

In [26]:
data.loc[data['date'].dt.month.isin(seasons['spring']), 'spring'] = 1
data.loc[data['date'].dt.month.isin(seasons['summer']), 'summber'] = 1
data.loc[data['date'].dt.month.isin(seasons['fall']), 'fall'] = 1
data.loc[data['date'].dt.month.isin(seasons['winter']), 'winter'] = 1

In [27]:
data.loc[data['winter']==1]

,longitude,latitude,name,id,date,temp,wind_direction,wind_speed,spring,summber,fall,winter
332,-112.82,53.05,CAMROSE,3011240,2012-12-02 16:00:00,-15.4,7.0,19.0,0,0,0,1
333,-112.82,53.05,CAMROSE,3011240,2012-12-04 05:00:00,-19.7,13.0,17.0,0,0,0,1
334,-112.82,53.05,CAMROSE,3011240,2012-12-04 18:00:00,-9.8,16.0,20.0,0,0,0,1
335,-112.82,53.05,CAMROSE,3011240,2012-12-09 11:00:00,-18.9,18.0,7.0,0,0,0,1
336,-112.82,53.05,CAMROSE,3011240,2012-12-09 14:00:00,-15.4,21.0,6.0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
228269,-115.55,51.19,BANFF CS,3050519,2023-02-28 03:00:00,-14.8,0.0,1.0,0,0,0,1
228270,-115.55,51.19,BANFF CS,3050519,2023-02-28 06:00:00,-13.4,22.0,5.0,0,0,0,1
228271,-115.55,51.19,BANFF CS,3050519,2023-02-28 13:00:00,-5.2,1.0,5.0,0,0,0,1
228272,-115.55,51.19,BANFF CS,3050519,2023-02-28 14:00:00,-4.0,4.0,5.0,0,0,0,1


In [28]:
#data = data.to_csv('Data\processed_ab_no_precp_test_with_season.csv', sep = ',', index = False)

In [9]:
grouped_data = data.groupby(['date', 'name']).size().unstack()

In [10]:
grouped_data

name,BANFF CS,BROOKS,CALGARY INTL A,CAMROSE,CLARESHOLM,CORONATION CLIMATE,DRUMHELLER EAST,EDMONTON INTL A,EDMONTON STONY PLAIN CS,LACOMBE CDA 2,LETHBRIDGE,LETHBRIDGE CDA,LLOYDMINSTER,MEDICINE HAT,MEDICINE HAT RCS,MILK RIVER,ONEFOUR CDA,ROCKY MTN HOUSE (AUT),STRATHMORE AGDM,VEGREVILLE
date,,,,,,,,,,,,,,,,,,,,
2012-07-09 17:00,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2012-07-09 18:00,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2012-07-09 19:00,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2012-07-10 10:00,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2012-07-10 11:00,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-14 06:00,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2023-09-14 07:00,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2023-09-14 18:00,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
